In [11]:
import torch.nn as nn
import cv2
from torchvision import transforms
import torch
import torchvision
import numpy as np
import ds_ear
import glob
from PIL import Image
from matplotlib import image
import sys
sys.path.append('../..')
from DLBio import pt_training
from torchvision.models.mobilenet import mobilenet_v2
from DLBio.pytorch_helpers import get_device, get_num_params
from DLBio.helpers import check_mkdir
from DLBio.pt_train_printer import Printer
import json
import matplotlib.pyplot as plt
from os.path import join



CATEGORIES = ["Falco","Konrad"]
RESIZE_Y = 150
RESIZE_X = 100
DATA_TEST_FOLDER = "../test/*jpg"


def get_data(folder):
    img_array = []
    img_array_resized = []
    files = glob.glob (folder)
    for idx, f in zip(range(len(files)),files):
        image = cv2.imread(f)
        img_array.append (image)
        img_array_resized.append(cv2.resize(img_array[idx],(RESIZE_Y,RESIZE_X)))
    return np.asarray(img_array_resized)


model = torch.load('./class_sample/model.pt')



In [13]:
image_array = []
files = glob.glob (DATA_TEST_FOLDER)
files.sort()
for idx, f in zip(range(len(files)),files):
    image = Image.open(f)
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Resize((RESIZE_Y, RESIZE_X)),
        torchvision.transforms.Lambda(lambda x: x.convert('RGB')),
        torchvision.transforms.ToTensor(),

        torchvision.transforms.Normalize(
            [0.49139968, 0.48215841, 0.44653091],
            [0.24703223, 0.24348513, 0.26158784]
        )
    ])
    image_transformed = transform(image)
    image_transformed = image_transformed.reshape(-1, RESIZE_Y, RESIZE_X, 1)
    image_transformed = image_transformed.permute(3, 0, 1, 2)
    image_array.append(image_transformed.type('torch.cuda.FloatTensor'))
# image_transformed = image_transformed.type('torch.cuda.FloatTensor')


In [15]:
NUMBER_AUTHORIZED = int(.7*len(image_array))

for i in image_array:
	with torch.no_grad():
		pred = model(i)
		pred = torch.softmax(pred, 1)
		pred = pred.cpu().numpy()
	classes_ = np.argmax(pred, 1)
	print(pred)
	print(classes_)
	# print(CATEGORIES[classes_[0]], "\n")
	temp_img = torch.squeeze(i).cpu().permute(1,2,0)
	plt.imshow( temp_img )
# counts = np.bincount(classes_)
# if np.max(counts) > NUMBER_AUTHORIZED:
# 	print("Welcome to your save room " + CATEGORIES[np.argmax(counts)] + "!")
# else: 
# 	print("Authentification Failed! You got no acces rights!")

[[2.08256868e-04 6.35882679e-05 4.91493056e-03 5.66407049e-04
  1.92165317e-03 2.62933038e-03 4.10186732e-03 8.41381625e-05
  5.00227034e-04 8.76283884e-01 4.84993029e-03 1.56702241e-04
  5.29320957e-03 4.53771463e-05 4.71713338e-06 3.69879672e-06
  1.14693925e-04 3.70703451e-02 4.34239373e-05 1.05507039e-04
  4.79854862e-05 2.02506967e-03 2.70715100e-04 5.54245780e-04
  1.97694717e-05 2.79728643e-04 6.42272353e-05 1.38018941e-05
  8.83804842e-06 9.12864834e-06 9.96558461e-03 3.68705732e-05
  9.05019639e-04 1.59055213e-04 6.02344342e-04 1.21887979e-05
  5.08736230e-05 1.27988154e-04 2.06786615e-04 5.50897357e-06
  2.69570978e-06 8.09037665e-05 2.49355780e-05 3.46133493e-05
  6.85495991e-07 3.47839014e-05 3.79243465e-06 1.91493658e-04
  9.67969572e-06 7.94715834e-06 1.27966189e-06 3.08079871e-05
  1.12165171e-05 2.27567343e-05 6.60870428e-05 4.63404067e-05
  6.70359659e-05 1.40984746e-04 9.53758956e-08 1.34202082e-06
  9.57546290e-05 1.07142523e-05 1.29138889e-05 6.71334919e-06
  2.1968

IndexError: list index out of range

In [ ]:
NUMBER_AUTHORIZED = int(.7*len(data_tensor))
with torch.no_grad():
	pred = model(data_tensor)
	pred = torch.softmax(pred, 1)
	pred = pred.cpu().numpy()

classes_ = np.argmax(pred, 1)
print(pred)
print(classes_)
counts = np.bincount(classes_)
if np.max(counts) > NUMBER_AUTHORIZED:
	print("Welcome to your save room " + CATEGORIES[np.argmax(counts)] + "!")
else: 
	print("Authentification Failed! You got no acces rights!")

In [ ]:
data = get_data(DATA_TEST_FOLDER)
data_tensor = torch.from_numpy(data)
data_tensor = data_tensor.permute(0, 3, 1, 2)
data_tensor = data_tensor.type('torch.cuda.FloatTensor')
#device = get_device()
#print(device)
#if device == cpu:
#    data_tensor = data_tensor.type('torch.FloatTensor')
#else:
#    data_tensor = data_tensor.type('torch.cuda.FloatTensor')
